In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
tf.reset_default_graph()

color_channels = 1  # data is in black and white, 1 channel
image_height = 28
image_width = 28

In [ ]:
from tensorflow.keras.datasets import mnist

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()  # sets up the data from mnist

# print(x_train.shape)  # contains 60,000 imgs that are 28x28
# print(y_train.shape)  # 60,000 values

In [ ]:
# TODO: Setting up Neural Network
class CNN:
    def __init__(self, image_height, image_width, channels, num_classes):
        self.input_layer = tf.placeholder(dtype=tf.float32, shape=[None, image_height, image_width, channels])
        
        conv_layer_1 = tf.layers.conv2d(self.input_layer, filters=32, kernel_size=[2, 2], padding="same", activation=tf.nn.relu)  # convolutional layer
        """The first input is the layer before the current one, the last input
            is the activation function. This act fn helps neural net solve for what
            the object could potentially be."""
        pooling_layer_1 = tf.layers.max_pooling2d(conv_layer_1, pool_size=[2, 2], strides=2)  # pooling layer
        
        conv_layer_2 = tf.layers.conv2d(pooling_layer_1, filters=32, kernel_size=[2, 2], padding="same", activation=tf.nn.relu)  # convolutional layer
        pooling_layer_2 = tf.layers.max_pooling2d(conv_layer_2, pool_size=[2, 2], strides=2)  # pooling layer
        
        # create the flatten layer...
        flattened_pooling = tf.layers.flatten(pooling_layer_2)  # pooling layer
        dense_layer = tf.layers.dense(flattened_pooling, 1024, activation=tf.nn.relu)  # dense layer: takes last layer, number of neurons to activate activation layer, and activation fn
        
        dropout = tf.layers.dropout(dense_layer, rate=0.4, training=True)  # dropout layer
        outputs = tf.layers.dense(dropout, num_classes)  # output layer
        
        
        # Getting data from the network
        self.choice = tf.argmax(outputs, axis=1)  # (1) CHOICE VARIABLE 
            # axis is dependent on which var to check for maximum, output is 1d, theres 1 axis
        self.probability = tf.nn.softmax(outputs)  # (2) SCALE PROBABILITIES
        self.labels = tf.placeholder(dtype=tf.float32, name="labels")  # (3) CREATE LABELS CONTAINER
        self.accuracy, self.accuracy_op = tf.metrics.accuracy(self.labels, self.choice)  # (4) GATHERING ACCURACY
        one_hot_labels = tf.one_hot(indices=tf.cast(self.labels, dtype=tf.int32), depth=num_classes)  # (5) ONE-HOT ENCODING, depth determined on num of possibilities (10)
        self.loss = tf.losses.softmax_cross_entropy(onehot_labels=one_hot_labels, logits=outputs)  # (6) LOSS FN
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-3)  # (7) OPTIMIZER, helps reduce loss with LEARNING RATE var (0>x>1)
        self.train_operation = optimizer.minimize(loss=self.loss, global_step=tf.train.get_global_step())  # (8) TRAINING STEP
        
        
        

In [ ]:
steps = 5000
batch_size = 32
num_classes = 10

test_img = x_test[2]
plt.imshow(test_img)
plt.show()
test_img = test_img.reshape(-1, image_height, image_width, 1)

x_train = x_train.reshape(-1, image_height, image_width, 1)

cnn = CNN(image_height, image_width, color_channels, num_classes)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    step = 0
    while step < steps:
        print(sess.run((cnn.train_operation, cnn.accuracy_op),
                       feed_dict={cnn.input_layer:x_train[step: step + batch_size],
                                cnn.labels:y_train[step: step + batch_size]}))
        step += batch_size
    
    print(sess.run(cnn.choice, feed_dict={cnn.input_layer:test_img}))
    